In [ ]:
import os.path

from tf2cv.model_provider import get_model as tf2cv_get_model
import tensorflow as tf
import numpy as np
import utils
from catboost import CatBoostClassifier

def extract_from_folder(root_dir, is_extract=False, network='efficientnet_b0'):
  image_paths, image_labels = utils.get_images(root_dir)

  images = tf.stack(list(map(utils.load_and_preprocess_image, image_paths)))
  labels = tf.stack(image_labels)

  if is_extract:
    net = tf2cv_get_model(network, pretrained=True, data_format="channels_last")
    images = net.features(images)

  return np.squeeze(images.numpy()), labels.numpy()

## 不同的划分比例

In [ ]:
for s in [0.1,0.3,0.2,0.4]:
  accu = []
  for seed in range(1,21):
    data_train = np.load(f'{s}dataset_{seed}_train_efficientnet_b0.npz')
    data_test = np.load(f'{s}dataset_{seed}_test_efficientnet_b0.npz')
    train_X, train_y = data_train['X'], data_train['y']
    test_X, test_y = data_test['X'], data_test['y']

    model = CatBoostClassifier(iterations=350, use_best_model=True, task_type='GPU')
    model.fit(train_X, train_y, eval_set=(test_X, test_y), verbose=False)
    accu.append(model.score(test_X, test_y) * 100)

  print(f'test_size={s}', np.mean(accu), np.std(accu), accu)      

## 不同的采样方法

In [ ]:
bay = []
mvs = []
ber = []

for seed in range(1,21):
    data_train = np.load(f'0.2dataset_{seed}_train_efficientnet_b0.npz')
    data_test = np.load(f'0.2dataset_{seed}_test_efficientnet_b0.npz')
    train_X, train_y = data_train['X'], data_train['y']
    test_X, test_y = data_test['X'], data_test['y']

    model = CatBoostClassifier(iterations=350, use_best_model=True, bootstrap_type='Bernoulli', task_type='GPU')
    model.fit(train_X, train_y, eval_set=(test_X, test_y), verbose=False)
    ber.append(model.score(test_X, test_y) * 100)

    model = CatBoostClassifier(iterations=350, use_best_model=True, bootstrap_type='MVS', task_type='GPU')
    model.fit(train_X, train_y, eval_set=(test_X, test_y), verbose=False)
    mvs.append(model.score(test_X, test_y) * 100)

    model = CatBoostClassifier(iterations=350, use_best_model=True, bootstrap_type='Bayesian', task_type='GPU')
    model.fit(train_X, train_y, eval_set=(test_X, test_y), verbose=False)
    bay.append(model.score(test_X, test_y) * 100)

print('Bernoulli: ', np.mean(ber), np.std(ber))
print('Bayesian: ', np.mean(bay), np.std(bay))
print('MVS: ', np.mean(mvs), np.std(mvs))

## 各指标具体结果

In [ ]:
from sklearn import metrics
from sklearn.metrics import auc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support as score

tprs=[]
aucs=[]
mean_fpr=np.linspace(0,1,100)

jianqie_precision = []
jianqie_recall = []
jianqie_fscore = []
zhangla_precision = []
zhangla_recall = []
zhangla_fscore = []

for seed in range(1,21):
    data_train = np.load(f'0.2dataset_{seed}_train_efficientnet_b0.npz')
    data_test = np.load(f'0.2dataset_{seed}_test_efficientnet_b0.npz')
    train_X, train_y = data_train['X'], data_train['y']
    test_X, test_y = data_test['X'], data_test['y']

    model = CatBoostClassifier(iterations=350, use_best_model=True, task_type='GPU')
    model.fit(train_X, train_y, eval_set=(test_X, test_y), verbose=False)

    print(classification_report(test_y, model.predict(test_X), target_names=['剪切', '张拉'], digits=5))

    precision, recall, fscore, support=score(test_y, model.predict(test_X))
    jianqie_precision.append(precision[0])
    jianqie_recall.append(recall[0])
    jianqie_fscore.append(fscore[0])
    zhangla_precision.append(precision[1])
    zhangla_recall.append(recall[1])
    zhangla_fscore.append(fscore[1])

    y_pred = model.predict_proba(test_X)
    y_pred = y_pred[:,1]
    fpr, tpr, _ = metrics.roc_curve(test_y, y_pred)
    tprs.append(np.interp(mean_fpr,fpr,tpr))
    tprs[-1][0]=0.0

    roc_auc=auc(fpr,tpr)
    aucs.append(roc_auc)

mean_tpr=np.mean(tprs, axis=0)

mean_tpr[-1]=1.0
mean_auc=auc(mean_fpr,mean_tpr) #计算平均AUC值
sns.set(rc={'figure.dpi': 300, 'axes.facecolor': 'WhiteSmoke'})
plt.plot(mean_fpr,mean_tpr,label=r'Mean ROC (area=%0.2f)'%mean_auc)
plt.plot([0, 1], [0, 1], color='orange', linestyle='--', label='Random')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(fancybox=True,shadow=True)
plt.savefig("roc_curve.jpg", bbox_inches='tight',dpi=500)

jianqie_precision = np.array(jianqie_precision) * 100
jianqie_recall = np.array(jianqie_recall) * 100
jianqie_fscore = np.array(jianqie_fscore) * 100
zhangla_precision = np.array(zhangla_precision) * 100
zhangla_recall = np.array(zhangla_recall) * 100
zhangla_fscore = np.array(zhangla_fscore) * 100

print(np.mean(jianqie_precision), np.std(jianqie_precision))
print(np.mean(jianqie_recall), np.std(jianqie_recall))
print(np.mean(jianqie_fscore), np.std(jianqie_fscore))
print(np.mean(zhangla_precision), np.std(zhangla_precision))
print(np.mean(zhangla_recall), np.std(zhangla_recall))
print(np.mean(zhangla_fscore), np.std(zhangla_fscore))

## 不同的特征提取模型

In [ ]:
network = ['efficientnet_b0', 'resnet50', 'densenet161']

for s in [0.2]:
    for net in network:
      accu = []
      for seed in range(1,21):
        data_train = np.load(f'{s}dataset_{seed}_train_{net}.npz')
        data_test = np.load(f'{s}dataset_{seed}_test_{net}.npz')
        train_X, train_y = data_train['X'], data_train['y']
        test_X, test_y = data_test['X'], data_test['y']

        model = CatBoostClassifier(iterations=350, use_best_model=True, verbose=False, task_type='GPU')
        model.fit(train_X, train_y, eval_set=(test_X, test_y), verbose=False)
        accu.append(model.score(test_X, test_y) * 100)
      print(f'test_size={s}, {net}', np.mean(accu), np.std(accu), accu)

## 不同的机器学习方法

In [ ]:
from sklearn import neighbors
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import  DecisionTreeClassifier

svm = []
knn = []
lr = []
gnb = []
rt = []

for seed in range(1,21):
    data_train = np.load(f'0.2dataset_{seed}_train_efficientnet_b0.npz')
    data_test = np.load(f'0.2dataset_{seed}_test_efficientnet_b0.npz')
    train_X, train_y = data_train['X'], data_train['y']
    test_X, test_y = data_test['X'], data_test['y']

    model = neighbors.KNeighborsClassifier(n_neighbors=3)
    model.fit(train_X, train_y)
    knn.append(model.score(test_X, test_y) * 100)

    model= LogisticRegression(max_iter=100)
    model.fit(train_X, train_y)
    lr.append(model.score(test_X, test_y) * 100)
    
    model = GaussianNB()
    model.fit(train_X, train_y)
    gnb.append(model.score(test_X, test_y) * 100)

    model= DecisionTreeClassifier(random_state=8)
    model.fit(train_X, train_y)
    rt.append(model.score(test_X, test_y) * 100)

print('KNN:', np.mean(knn), np.std(knn))
print('LR:', np.mean(lr), np.std(lr))
print('GNB:', np.mean(gnb), np.std(gnb))
print('DT:', np.mean(rt), np.std(rt))

## 深度学习特征提取的有效性

### 声谱图特征

In [ ]:
data = np.load('声音数据集no_extract.npz')
X, y = data['X'], data['y']
X = X.reshape(161, -1)

for s in [0.2]:
  accu = []
  for seed in range(1,6):
    train_aug_X = []
    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=s, random_state=seed, shuffle=True)
    model = CatBoostClassifier(iterations=450, task_type="GPU", use_best_model=True, verbose=False)
    model.fit(train_X, train_y, eval_set=(test_X, test_y), verbose=False)
    accu.append(model.score(test_X, test_y) * 100)

  print(f'test_size={s}', np.mean(accu), np.std(accu), accu)

### PCA降维后特征

In [ ]:
from sklearn.decomposition import PCA

data = np.load('声音数据集no_extract.npz')
X, y = data['X'], data['y']
X = X.reshape(161, -1)

n_components = 161

pca = PCA(n_components=n_components)
X = pca.fit_transform(X)

for s in [0.2]:
  accu = []
  for seed in range(1,6):
    train_aug_X = []
    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=s, random_state=seed, shuffle=True)
    model = CatBoostClassifier(iterations=700, use_best_model=True, verbose=False, task_type="GPU")
    model.fit(train_X, train_y, eval_set=(test_X, test_y), verbose=False)
    accu.append(model.score(test_X, test_y) * 100)

  print(f'test_size={s}', np.mean(accu), np.std(accu), accu)

### 深度学习提取的特征

In [ ]:
data = np.load('声音数据集.npz')
X, y = data['X'], data['y']

for s in [0.2]:
  accu = []
  for seed in range(1,6):
    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=s, random_state=seed, shuffle=True)
    model = CatBoostClassifier(iterations=350, task_type="GPU", use_best_model=True, verbose=False)
    model.fit(train_X, train_y, eval_set=(test_X, test_y), verbose=False)
    accu.append(model.score(test_X, test_y) * 100)

  print(f'test_size={s}', np.mean(accu), np.std(accu), accu)